In [ ]:
import pandas as pd
import joblib

from pathlib import Path
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# === Data Preprocessing Starts Here === #

In [ ]:
# === CONFIG === #
project = Path.cwd().parent
target_folder = project / "0 - data"
add_folder = project / "4 - deployment" 
file_path = target_folder / "Task_Automation_Dataset.xlsx"
sheet = "Task_Automation_Dataset"

df_raw = pd.read_excel(file_path, sheet_name=sheet)
df_work = df_raw.copy(deep=True)

target_col = "Automation Suitable"

def print_table_nice(df, title):
    print(f"\n===== {title} =====")
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

def get_class_variables_summary(df, target_col=None, unique_threshold=20, force_interval=None):
    """Return a SAS-like summary for class variables, excluding forced interval columns."""
    import pandas as pd
    
    force_interval = set(force_interval or [])
    
    class_vars = [
        col for col in df.columns
        if col not in force_interval and 
           (df[col].dtype == 'object' or df[col].nunique() <= unique_threshold)
    ]
    
    summary = []
    for col in class_vars:
        role = "TARGET" if col == target_col else "INPUT"
        levels = df[col].nunique(dropna=True)
        missing = df[col].isna().sum()
        
        mode = df[col].mode().iloc[0] if not df[col].mode().empty else None
        mode_pct = (df[col] == mode).mean() * 100 if mode is not None else None
        
        mode2 = None
        mode2_pct = None
        if levels > 1:
            counts = df[col].value_counts(normalize=True) * 100
            if len(counts) > 1:
                mode2 = counts.index[1]
                mode2_pct = counts.iloc[1]
        
        summary.append([role, col, levels, missing, mode, mode_pct, mode2, mode2_pct])
    
    return pd.DataFrame(summary, columns=[
        "Role", "Variable Name", "Levels", "Missing", 
        "Mode", "Mode Percentage", "Mode2", "Mode2 Percentage"
    ])

def get_interval_variables_summary(df, target_col=None, unique_threshold=20, force_interval=None):
    """Return a SAS-like summary for interval variables with numeric stats."""
    import pandas as pd
    
    force_interval = set(force_interval or [])
    
    interval_vars = [
        col for col in df.columns
        if col in force_interval or (
            pd.api.types.is_numeric_dtype(df[col]) and df[col].nunique() > unique_threshold
        )
    ]
    
    summary = []
    for col in interval_vars:
        series = df[col].dropna()
        role = "TARGET" if col == target_col else "INPUT"
        
        mean_val = series.mean()
        std_dev = series.std()
        missing = df[col].isna().sum()
        min_val = series.min()
        median_val = series.median()
        max_val = series.max()
        skew_val = series.skew()
        kurt_val = series.kurtosis()
        
        summary.append([
            col, role, mean_val, std_dev, missing,
            min_val, median_val, max_val, skew_val, kurt_val
        ])
    
    return pd.DataFrame(summary, columns=[
        "Variable", "Role", "Mean", "Standard Deviation",
        "Missing", "Minimum", "Median", "Maximum",
        "Skewness", "Kurtosis"
    ])

def summary_statistics(df, target_col=None):
    force_interval = ["Complexity (1-5)"]

    class_summary = get_class_variables_summary(
        df,
        target_col=target_col,
        force_interval=force_interval
    )
    
    interval_summary = get_interval_variables_summary(
        df,
        target_col=target_col,
        force_interval=force_interval
    )

    print()
    print_table_nice(class_summary, "CLASS VARIABLES SUMMARY")
    print_table_nice(interval_summary, "INTERVAL VARIABLES SUMMARY")
    print()

In [ ]:
# === Outliers Functionality === #
def detect_outliers_iqr(df, col):
    series = pd.to_numeric(df[col], errors='coerce')
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    mask = (series < lower_bound) | (series > upper_bound)
    print(f"[{col}] Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}, Outliers: {mask.sum()}")
    return mask, lower_bound, upper_bound

def fix_outliers_clip(df, col):
    mask, lower, upper = detect_outliers_iqr(df, col)
    df[col] = pd.to_numeric(df[col], errors='coerce').clip(lower, upper)
    print(f"[{col}] Outliers clipped to range {lower:.2f}–{upper:.2f}")
    return df

def run_outliers(df_work):
    # Clip outliers for Time Taken (mins)
    df_work = fix_outliers_clip(df_work, "Time Taken (mins)")

    # Clip outliers for Error Rate (%) and also ensure 0–100%
    df_work["Error Rate (%)"] = df_work["Error Rate (%)"].clip(0, 100)

    return df_work

In [ ]:
# === Invalid Entry Handling Functionality === #
NUM_WORDS = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
    "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
    "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14,
    "fifteen": 15, "sixteen": 16, "seventeen": 17, "eighteen": 18,
    "nineteen": 19, "twenty": 20, "thirty": 30, "forty": 40,
    "fifty": 50, "sixty": 60, "seventy": 70, "eighty": 80, "ninety": 90,
    "hundred": 100
}

def words_to_num(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    if text.replace(".", "", 1).isdigit():
        return float(text)
    words = text.split()
    total = 0
    for w in words:
        if w in NUM_WORDS:
            total += NUM_WORDS[w]
    return total if total > 0 else None

def fix_invalid_entries(df): # Fixed 2
    # --- Time Taken (mins) ---
    df["Time Taken (mins)"] = df["Time Taken (mins)"].apply(words_to_num)

    # --- Complexity (1-5) ---
    df["Complexity (1-5)"] = df["Complexity (1-5)"].apply(words_to_num)
    df["Complexity (1-5)"] = pd.to_numeric(df["Complexity (1-5)"], errors='coerce')
    df["Complexity (1-5)"] = df["Complexity (1-5)"].where(df["Complexity (1-5)"].between(1, 5), pd.NA)

    # Fill missing Complexity with mode
    if df["Complexity (1-5)"].isna().any():
        mode_value = df["Complexity (1-5)"].mode().iloc[0]
        df["Complexity (1-5)"].fillna(mode_value, inplace=True)
    
    # --- Error Rate (%) ---
    df["Error Rate (%)"] = df["Error Rate (%)"].astype(str).str.strip().str.replace("%", "", regex=False)
    df["Error Rate (%)"] = pd.to_numeric(df["Error Rate (%)"], errors="coerce")
    df["Error Rate (%)"] = df["Error Rate (%)"].where(df["Error Rate (%)"].between(0, 100), pd.NA)

    print("Invalid entries fixed: numeric conversions, domain limits, and missing values handled.")
    return df

In [ ]:
# === Missing Values Functionality === #
def handle_missing_values(df):
    # Time Taken (mins)
    time_median = df["Time Taken (mins)"].median()
    df["Time Taken (mins)"].fillna(time_median, inplace=True)
    print(f"[Time Taken (mins)] Missing filled with median = {time_median}")

    # Frequency
    freq_mode = df["Frequency"].mode()[0]
    df["Frequency"].fillna(freq_mode, inplace=True)
    print(f"[Frequency] Missing filled with mode = {freq_mode}")

    # Department
    dept_mode = df["Department"].mode()[0]
    df["Department"].fillna(dept_mode, inplace=True)
    print(f"[Department] Missing filled with mode = {dept_mode}")

    # Error Rate (%)
    err_mean = df["Error Rate (%)"].mean()
    df["Error Rate (%)"].fillna(err_mean, inplace=True)
    print(f"[Error Rate (%)] Missing filled with mean = {err_mean}")

    # Rule-Based Indicator
    RBI_mode = df["Rule-Based Indicator"].mode()[0]
    df["Rule-Based Indicator"].fillna(RBI_mode, inplace=True)
    print(f"[Rule-Based Indicator] Missing filled with mode = {RBI_mode}")

    # Process Stability
    stable_mode = df["Process Stability"].mode()[0]
    df["Process Stability"].fillna(stable_mode, inplace=True)
    print(f"[Process Stability] Missing filled with mode = {stable_mode}")

    # Data Structure
    DS_mode = df["Data Structure"].mode()[0]
    df["Data Structure"].fillna(DS_mode, inplace=True)
    print(f"[Data Structure] Missing filled with mode = {DS_mode}")

    return df

In [ ]:
# === Inconsistent Data Formatting Functionality === #
def fix_typographical_errors(df):
    tool_map = {
        "Excel": "Excel", "MS Excel": "Excel", "EXCEL": "Excel",
        "Power BI": "Power BI", "Power Bi": "Power BI",
        "SharePoint": "SharePoint", "SHAREPOINT": "SharePoint",
        "Outlook": "Outlook", "MS Outlook": "Outlook",
        "Microsoft Teams": "Microsoft Teams", "MS Teams": "Microsoft Teams",
        "Google Sheets": "Google Sheets", "Google SHEETS": "Google Sheets",
        "SAP": "SAP", "Zoom": "Zoom", "Jira": "Jira", "Slack": "Slack",
        "Trello": "Trello", "Custom Portal": "Custom Portal"
    }
    dept_map = {
        "IT": "IT", "hr": "Human Resource", "HR": "Human Resource", "human resource": "Human Resource",
        "Sales": "Sales", "SALES": "Sales", "Marketing": "Marketing", "Compliance": "Compliance",
        "Finance": "Finance", "Operations": "Operations", "Procurement": "Procurement",
        "Admin": "Admin", "ADMIN": "Admin", "Customer Service": "Customer Service"
    }
    freq_map = {
        "Daily": "Daily", "DAILY": "Daily",
        "Weekly": "Weekly",
        "Monthly": "Monthly", "MONTHLY": "Monthly",
        "Quarterly": "Quarterly",
        "Ad-Hoc": "Ad-Hoc"
    }
    rbi_map = {
        "YES": "Yes",
        "NO": "No"
    }
    stable_map = {
        "HIGH": "High",
        "LOW": "Low"
    }
    ds_map = {
        "Not-structured": "Unstructured",
        "SEMI-STRUCTURED": "Semi-Structured",
        "CLEAN": "Structured", "structured": "Structured"
    }

    # Safe mapping with NaN handling
    for col, mapping in {
        "Tool Used": tool_map,
        "Department": dept_map,
        "Frequency": freq_map,
        "Rule-Based Indicator": rbi_map,
        "Process Stability": stable_map,
        "Data Structure": ds_map
    }.items():
        df[col] = df[col].apply(lambda x: mapping.get(str(x).strip(), x) if pd.notna(x) else x)

    print("Typographical errors fixed: standardized Tool Used, Department, Frequency, Rule-Based Indicator, Process Stability, Data Structure")
    return df

In [ ]:
# === Data Split and Standardization Functionality === #
def split_and_standardize(df, target_col, test_size=0.2, random_state=42):
    cols_to_standardize = ["Time Taken (mins)", "Error Rate (%)"]

    # Split the dataset
    train_df, test_df = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=df[target_col]  # keeps class balance similar
    )

    # Fit scaler on train_data only
    scaler = StandardScaler()
    train_df[cols_to_standardize] = scaler.fit_transform(train_df[cols_to_standardize])
    test_df[cols_to_standardize] = scaler.transform(test_df[cols_to_standardize])

    # Save scaler for deployment
    joblib.dump(scaler, add_folder / "scaler.pkl")
    print("Scaler saved to: 4 - deployment/scaler.pkl")

    return train_df, test_df

In [ ]:
# === Execute Data Preprocessing Pipeline === #
def preprocess_pipeline(df, save_path= target_folder / "Task_Automation_Dataset_Clean.xlsx"):
    # Step 1: Fix invalid entries
    df = fix_invalid_entries(df)
    
    # Step 2: Handle missing values
    df = handle_missing_values(df)
    
    # Step 3: Fix typographical errors
    df = fix_typographical_errors(df)
    
    # Step 4: Handle outliers
    df = run_outliers(df)
    
    # Step 5: Split dataset + standardize using train_data only
    train_df, test_df = split_and_standardize(df, target_col)
    
    # Step 6: Summary statistics
    summary_statistics(df, target_col)
    
    train_df.to_excel(target_folder / "train_clean.xlsx", index=False)
    test_df.to_excel(target_folder / "test_clean.xlsx", index=False)
    print("Saved: 0 - data/train_clean.xlsx and 0 - data/test_clean.xlsx")

    df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
    df.to_excel(save_path, index=False)
    print(f"\nCleaned dataset saved to: {save_path}")

    return train_df, test_df

train_df, test_df = preprocess_pipeline(df_raw)

# === Data Preprocessing End Here === #

C:\Users\victo\AppData\Local\Temp\ipykernel_42464\1811030218.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Complexity (1-5)"].fillna(mode_value, inplace=True)
C:\Users\victo\AppData\Local\Temp\ipykernel_42464\2533215133.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


Invalid entries fixed: numeric conversions, domain limits, and missing values handled.
[Time Taken (mins)] Missing filled with median = 91.0
[Frequency] Missing filled with mode = Monthly
[Department] Missing filled with mode = Compliance
[Error Rate (%)] Missing filled with mean = 5.227835051546392
[Rule-Based Indicator] Missing filled with mode = No
[Process Stability] Missing filled with mode = Medium
[Data Structure] Missing filled with mode = Structured
Typographical errors fixed: standardized Tool Used, Department, Frequency, Rule-Based Indicator, Process Stability, Data Structure
[Time Taken (mins)] Lower: -87.00, Upper: 275.00, Outliers: 7
[Time Taken (mins)] Outliers clipped to range -87.00–275.00
Scaler saved to: 4 - deployment/scaler.pkl


===== CLASS VARIABLES SUMMARY =====
+--------+----------------------+----------+-----------+---------------------------+-------------------+------------------+--------------------+
| Role   | Variable Name        |   Levels |   Missing | M